In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorpack.utils import logger

from tgan.data import load_data
from tgan.model import TUNABLE_VARIABLES, TGANModel
from tgan.evaluation import evaluate_classification

In [2]:
def prepare_hyperparameter_search(steps_per_epoch, num_random_search):
    model_kwargs = []
    basic_kwargs = {
        'max_epoch': 1,
        'steps_per_epoch': steps_per_epoch,
        'restore_session': False
    }
    for i in range(num_random_search):
        kwargs = {name: np.random.choice(choices) for name, choices in TUNABLE_VARIABLES.items()}
        kwargs.update(basic_kwargs)
        model_kwargs.append(kwargs)

    return model_kwargs

In [3]:
name = 'experiment'
max_epoch = 1
steps_per_epoch = 5
output_epoch = 1
sample_rows = 1000
file_path = 'data/census-train.csv'
continuous_columns = [0, 5, 16, 17, 18, 29, 38]
num_random_search = 5
original_data = pd.read_csv(file_path)

In [4]:
# Load and split data
test_data, train_data = train_test_split(original_data, train_size=0.8, shuffle=True)

/home/xino/.virtualenvs/tgan_mit/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
# Prepare hyperparameter search
model_kwargs = prepare_hyperparameter_search(steps_per_epoch, num_random_search)

In [6]:
# Training models and sampling data
synthesized_data = []
for index, kwargs in enumerate(model_kwargs):
    logger.info('Training TGAN Model %d/%d', index + 1, num_random_search)
    
    tf.reset_default_graph()
    model = TGANModel(continuous_columns, output='{}/model_{}'.format(name, index), **kwargs)
    model.fit(train_data)
    synthesized_data.append(model.sample(sample_rows))


[0404 00:21:18 @<ipython-input-6-ebf7691fed61>:3] Training TGAN Model 1/5
[0404 00:21:20 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[0404 00:21:20 @registry.py:126] gen/LSTM/00/FC input: [50, 400]
[0404 00:21:20 @registry.py:134] gen/LSTM/00/FC output: [50, 600]
[0404 00:21:20 @registry.py:126] gen/LSTM/00/FC2 input: [50, 600]
[0404 00:21:20 @registry.py:134] gen/LSTM/00/FC2 output: [50, 1]
[0404 00:21:20 @registry.py:126] gen/LSTM/01/FC input: [50, 400]
[0404 00:21:20 @registry.py:134] gen/LSTM/01/FC output: [50, 600]
[0404 00:21:20 @registry.py:126] gen/LSTM/01/FC2 input: [50, 600]
[0404 00:21:20 @registry.py:134] gen/LSTM/01/FC2 output: [50, 5]
[0404 00:21:20 @registry.py:126] gen/LSTM/01/FC3 input: [50, 5]
[0404 00:21:21 @registry.py:13

100%|##########|5/5[00:21<00:00, 0.19it/s]

[0404 00:22:17 @base.py:285] Epoch 1 (global_step 5) finished, time:21.7 seconds.


[0404 00:22:18 @saver.py:79] Model saved to experiment/model_0/model/model-5.
[0404 00:22:18 @monitor.py:467] QueueInput/queue_size: 50
[0404 00:22:18 @monitor.py:467] train/GAN_loss/discrim/accuracy_fake: 0.4
[0404 00:22:18 @monitor.py:467] train/GAN_loss/discrim/accuracy_real: 0.6
[0404 00:22:18 @monitor.py:467] train/GAN_loss/discrim/loss: 0.93679
[0404 00:22:18 @monitor.py:467] train/GAN_loss/gen/final-g-loss: 28.347
[0404 00:22:18 @monitor.py:467] train/GAN_loss/gen/klloss: 27.156
[0404 00:22:18 @monitor.py:467] train/GAN_loss/gen/loss: 1.1903
[0404 00:22:18 @base.py:289] Training has finished!
[0404 00:22:18 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.
[0404 00:22:24 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0404 00:22:24 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0404 00:22:24 @sessinit.py:87] WRN The following variables are in the checkpoint, but not

 10%|9         |19/200[00:09<01:27, 2.06it/s]

...

[0404 00:26:30 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[0404 00:26:35 @registry.py:126] discrim/dis_fc4/fc input: [100, 210]
[0404 00:26:35 @registry.py:134] discrim/dis_fc4/fc output: [100, 200]
[0404 00:26:35 @registry.py:126] discrim/dis_fc4/fc_diversity input: [100, 200]
[0404 00:26:35 @registry.py:134] discrim/dis_fc4/fc_diversity output: [100, 100]
[0404 00:26:49 @logger.py:90] Argv: /home/xino/.virtualenvs/tgan_mit/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1000/jupyter/kernel-60e1ed5e-3e1b-45c7-a833-c63b6fba9ca2.json
[0404 00:26:50 @model_utils.py:67] List of Trainable Variables: 
name                              shape           #elements
--------------------------------  ------------  -----------
gen/LSTM/go:0                     [1, 300]              300
gen/LSTM/lstm_cell/kernel:0       [1000, 1200]      1200000
gen/LSTM/lstm_cell/bias:0         [1200]               1200
gen/LSTM/00/FC/W:0              

100%|##########|5/5[00:23<00:00, 0.15it/s]

[0404 00:27:24 @base.py:285] Epoch 1 (global_step 5) finished, time:23.9 seconds.


[0404 00:27:24 @saver.py:79] Model saved to experiment/model_4/model/model-5.
[0404 00:27:24 @monitor.py:467] QueueInput/queue_size: 50
[0404 00:27:24 @monitor.py:467] train/GAN_loss/discrim/accuracy_fake: 0.21
[0404 00:27:24 @monitor.py:467] train/GAN_loss/discrim/accuracy_real: 0.74
[0404 00:27:24 @monitor.py:467] train/GAN_loss/discrim/loss: 0.97704
[0404 00:27:24 @monitor.py:467] train/GAN_loss/gen/final-g-loss: 19.553
[0404 00:27:24 @monitor.py:467] train/GAN_loss/gen/klloss: 18.779
[0404 00:27:24 @monitor.py:467] train/GAN_loss/gen/loss: 0.77378
[0404 00:27:24 @base.py:289] Training has finished!
[0404 00:27:24 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.
[0404 00:27:30 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0404 00:27:30 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0404 00:27:30 @sessinit.py:87] WRN The following variables are in the checkpoint, but 

  9%|9         |9/100[00:04<00:39, 2.30it/s]


In [ ]:
for index, dataset in enumerate(synthesized_data):
    dataset.to_csv('{}.csv'.format(index), index=False, header=True)

In [6]:
if 'synthesized_data' not in locals():
    synthesized_data = []
    for i in range(num_random_search):
        print(i)
        synthesized_data.append(pd.read_csv('{}.csv'.format(i)))

0
1
2
3
4


In [8]:
# Evaluating synthesized data
for index, sampled_train_data in enumerate(synthesized_data):
    print(index)
    model_kwargs[index]['score'] = evaluate_classification(sampled_train_data, test_data, continuous_columns)

/home/xino/Pythia/MIT/TGAN/tgan/evaluation.py:61: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  dataset = pd.concat([train_data, test_data])


MemoryError: 

In [2]:
import pandas as pd
import numpy as np

In [6]:
df = pd.DataFrame(np.full((5, 2), 7))
df

,0,1
0,7,7
1,7,7
2,7,7
3,7,7
4,7,7


In [7]:
df[1]

0    7
1    7
2    7
3    7
4    7
Name: 1, dtype: int64

In [11]:
df.iloc[:, 1]

0    7
1    7
2    7
3    7
4    7
Name: 1, dtype: int64